In [1]:
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
import json

In [3]:

dataset = pd.read_csv("dataset.csv")
dataset

,label,author,comment,id,channel,title
0,1,Abatistalima-b4t,"AЕR𝐎𝟴𝟴 gua main dikit aja, saldo lngsg nambah",CIUnp8NMw8w,Deddy Corbuzier,TERNYATA ARAFAH PERNAH PACARAN SAMA ATE‼️😅 - P...
1,1,AbdulkadirMustafa-z4t,Sebelumnya bermain di tempat lain terasa kuran...,CIUnp8NMw8w,Deddy Corbuzier,TERNYATA ARAFAH PERNAH PACARAN SAMA ATE‼️😅 - P...
2,1,abdullaGodgift-m1w,AЕR𝐎𝟴𝟴 aku sih gmpang bgt dapet WEDEY gede dsini,CIUnp8NMw8w,Deddy Corbuzier,TERNYATA ARAFAH PERNAH PACARAN SAMA ATE‼️😅 - P...
3,1,AbdullahFABIAN-i2m,"AЕR𝐎𝟴𝟴 gua cba bentar, eh lgnsung dapet jackpot",CIUnp8NMw8w,Deddy Corbuzier,TERNYATA ARAFAH PERNAH PACARAN SAMA ATE‼️😅 - P...
4,1,Abergerontorres-s3d,"AЕR𝐎𝟴𝟴 aku cba maen, malah jackpot gede",CIUnp8NMw8w,Deddy Corbuzier,TERNYATA ARAFAH PERNAH PACARAN SAMA ATE‼️😅 - P...
...,...,...,...,...,...,...
20802,0,HolleSangma,gua udah 5tahun main di alexis17. jangka pan...,sck5d_eDT30,Jonathan Liandi,Review Skin Collector Melissa - Mobile Legends
20803,0,Jigen287,Kita butuh impaction,sck5d_eDT30,Jonathan Liandi,Review Skin Collector Melissa - Mobile Legends
20804,0,mirzarayyan5423,"Cih, masih epigh",sck5d_eDT30,Jonathan Liandi,Review Skin Collector Melissa - Mobile Legends
20805,0,jayyjoo15,SKIN GARA UDAH KELIATAN BANGG,sck5d_eDT30,Jonathan Liandi,Review Skin Collector Melissa - Mobile Legends


In [4]:
df = pd.DataFrame(dataset['comment'])

In [14]:
from transformers import GPT2Tokenizer
from sklearn.model_selection import train_test_split

# Loading the dataset to train our model
dataset = pd.read_csv("dataset.csv")

# Split the dataset into train and test sets
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

# Initialize tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Modified tokenize function
def tokenize_function(text):
    return tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=128,  
        return_tensors="pt"
    )

# Apply tokenization to both train and test sets
train_tokenized = train_data['comment'].apply(tokenize_function)
test_tokenized = test_data['comment'].apply(tokenize_function)


c:\Users\diash\anaconda3\envs\Generative_ai\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [16]:
from transformers import GPT2ForSequenceClassification

model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Using Gemma
from transformers import GemmaForSequenceClassification, GemmaTokenizer

tokenizer = GemmaTokenizer.from_pretrained("google/gemma-2b")
model = GemmaForSequenceClassification.from_pretrained("google/gemma-2b", num_labels=2)

# Or using BERT
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Or using RoBERTa
from transformers import RobertaTokenizer, RobertaForSequenceClassification

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# Or using DeBERTa
from transformers import DebertaTokenizer, DebertaForSequenceClassification

tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')
model = DebertaForSequenceClassification.from_pretrained('microsoft/deberta-base', num_labels=2)

In [17]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [18]:
from transformers import TrainingArguments, Trainer
from torch.utils.data import Dataset

# Create a custom dataset class
class CommentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts.iloc[idx])
        label = self.labels.iloc[idx]
        
        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        # Remove the batch dimension the tokenizer adds
        item = {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': label
        }
        
        return item

# Create dataset instances
train_dataset = CommentDataset(
    train_data['comment'], 
    train_data['label'],  # Assuming 'label' is your target column
    tokenizer
)

eval_dataset = CommentDataset(
    test_data['comment'],
    test_data['label'],  # Assuming 'label' is your target column
    tokenizer
)

# Create small subsets for testing
small_train_dataset = CommentDataset(
    train_data['comment'].iloc[:10],
    train_data['label'].iloc[:10],  # Assuming 'label' is your target column
    tokenizer
)

small_eval_dataset = CommentDataset(
    test_data['comment'].iloc[:10],
    test_data['label'].iloc[:10],  # Assuming 'label' is your target column
    tokenizer
)

# Training arguments and trainer setup remain the same
training_args = TrainingArguments(
    output_dir="test_trainer",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,  # Add number of training epochs
    logging_dir="./logs",
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()


                                     
100%|██████████| 6/6 [00:36<00:00,  6.07s/it]

{'train_runtime': 36.4253, 'train_samples_per_second': 0.824, 'train_steps_per_second': 0.165, 'train_loss': 3.8675966262817383, 'epoch': 2.4}


TrainOutput(global_step=6, training_loss=3.8675966262817383, metrics={'train_runtime': 36.4253, 'train_samples_per_second': 0.824, 'train_steps_per_second': 0.165, 'total_flos': 1567780503552.0, 'train_loss': 3.8675966262817383, 'epoch': 2.4})

In [19]:
import evaluate

trainer.evaluate()


100%|██████████| 10/10 [00:03<00:00,  3.21it/s]


{'eval_loss': 1.1046736240386963,
 'eval_accuracy': 0.8,
 'eval_runtime': 3.5205,
 'eval_samples_per_second': 2.841,
 'eval_steps_per_second': 2.841,
 'epoch': 2.4}

In [24]:
pip install seaborn

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\diash\anaconda3\envs\generative_ai\lib\site-packages\trt_pose-0.0.1-py3.11-win-amd64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
